<a href="https://colab.research.google.com/github/EhsanEs-hub/BlockChain/blob/master/CryptoZombieFront_End.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
<!DOCTYPE html>
<html lang="en">
  <head>
    <meta charset="UTF-8">
    <title>CryptoZombies front-end</title>
    <script language="javascript" type="text/javascript" src="https://cdnjs.cloudflare.com/ajax/libs/jquery/3.3.1/jquery.min.js"></script>
    <script language="javascript" type="text/javascript" src="web3.min.js"></script>
	  <script language="javascript" type="text/javascript" src="cryptozombies_abi.js"></script>
    // we can import the ABI definition into our project.
  </head>
  <body>
    <script>

      // use this variable to store our instantiated contract.
      var cryptoZombies;
      
      var userAccount;
      
      function startApp() {
      
        var cryptoZombiesAddress = "YOUR_CONTRACT_ADDRESS";
        cryptoZombies = new web3js.eth.Contract(cryptoZombiesABI, cryptoZombiesAddress);
/*        
We can see which account is currently active onthe injected web3 variable via:
var userAccount = web3.eth.accounts[0]
*/
        var accountInterval = setInterval(function() { 
            
            // Check if account has changed
          if (web3.eth.accounts[0] !== userAccount) {
            userAccount = web3.eth.accounts[0]

            // Call a function to update the UI with the new account
            getZombiesByOwner(userAccount);
            .then(displayZombies);
          }
        }, 100);
 /* 
this function check every 100 milliseconds to see if userAccount is still equal web3.eth.accounts[0]
(i.e. does the user still have that account active). If not, it reassigns userAccount to the currently
active account, and calls a function to update the display. 
*/

/* to filter events and only listen for changes related to the current user,
our Solidity contract would have to use the indexed keyword,
because _from and _to are indexed, we can filter for them in our event listener in our front end.
*/    
        cryptoZombies.events.Transfer({ filter: { _to: userAccount } })
        // Use `filter` to only fire this code when `_to` equals `userAccount`

        .on("data", function(event) {
                  
          let data = event.returnValues;
          getZombiesByOwner(userAccount).then(displayZombies);
          
        }).on("error", console.error);
      
      }
            
      function displayZombies(ids) {
        
        $("#zombies").empty();
        
        for (id of ids) {
          
        // Look up zombie details from our contract. Returns a `zombie` object
          getZombieDetails(id)
          .then(function(zombie) {
              
          // Using ES6 "template literals" to inject variables into the HTML.
          // Append each one to our #zombies div
            $("#zombies").append(`<div class="zombie">
              <ul>
                <li>Name: ${zombie.name}</li>
                <li>DNA: ${zombie.dna}</li>
                <li>Level: ${zombie.level}</li>
                <li>Wins: ${zombie.winCount}</li>
                <li>Losses: ${zombie.lossCount}</li>
                <li>Ready Time: ${zombie.readyTime}</li>
              </ul>
            </div>`);
          });
        }
      }

      function createRandomZombie(name) {

        // This is taking a while, so update the UI to let the user know the transaction has been sent.
        $("#txStatus").text("Creating new zombie on the blockchain. This may take a while...");

        // Send the tx to our contract:
        return cryptoZombies.methods.createRandomZombie(name)
        .send({ from: userAccount })
        .on("receipt", function(receipt) {
          $("#txStatus").text("Successfully created " + name + "!");

        // Transaction was accepted into the blockchain, lets redraw the UI
          getZombiesByOwner(userAccount).then(displayZombies);
        })
        .on("error", function(error) {

          // Do something to alert the user their transaction has failed
          $("#txStatus").text(error);
        });
      }
      
      function feedOnKitty(zombieId, kittyId) {
        $("#txStatus").text( "Eating a kitty. This may take a while...");
        return cryptoZombies.methods.feedOnKitty(zombieId, kittyId)
        .send({ from: userAccount })
        .on("receipt", function(receipt) {
          $("#txStatus").text("Ate a kitty and spawned a new Zombie!");
          getZombiesByOwner(userAccount).then(displayZombies);
        })
        .on("error", function(error) {
          $("#txStatus").text(error);
        });
      }

      function levelUp(zombieId) {
      
        $("#txStatus").text("Leveling up your zombie...");
        return cryptoZombies.methods.levelUp(zombieId)

// Web3.js has a conversion utility that will convert 1 ETH to Wei   
        .send({ from: userAccount, value: web3js.utils.toWei("0.001", "ether")})
        .on("receipt", function(receipt) {  
          $("#txStatus").text("Power overwhelming! Zombie successfully leveled up");
/*
We dont need to redraw the UI by querying our smart contract with getZombiesByOwner
because in this case we know the only thing that is changed is the one zombies level.
*/
        })
        .on("error", function(error) {
          $("#txStatus").text(error);
        });
      }		

      function getZombieDetails(id) {
        return cryptoZombies.methods.zombies(id).call()
      }
      
      function zombieToOwner(id) {
        return cryptoZombies.methods.zombieToOwner(id).call()
      }

// Use getZombiesByOwner(owner) function to look up all the IDs of zombies the current user owns. 
      function getZombiesByOwner(owner) {
        return cryptoZombies.methods.getZombiesByOwner(owner).call()
      }
      
      window.addEventListener('load', startApp() {

        // Checking if Web3 has been injected by the browser (Mist/MetaMask)
        if (typeof web3 !== 'undefined') {
            
          // Use Mist/MetaMask's provider
          web3js = new Web3(web3.currentProvider);

        } else {
            // Handle the case where the user doesn't have Metamask installed
          // Probably show them a message prompting them to install Metamask
        }

        // Now start app & access web3 freely:
        startApp()

      })
    </script>
  </body>
</html>
